
# Advanced ML: Working with an embedding vectorizer


#### 1. install embedding vectorizer:
pip install embeddingvectorizer

https://github.com/ccs-amsterdam/embeddingvectorizer

#### 2. download the dataset:
https://surfdrive.surf.nl/files/index.php/s/HKR33cTie8NT6Zh
    
#### 3. Download a pre-trained embedding model:
https://surfdrive.surf.nl/files/index.php/s/tvKQaskomfg0n6J


In [3]:
import embeddingvectorizer
import pandas as pd
import gensim
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_pickle('dataset_vermeer.pkl')
X_train, X_test, y_train, y_test = train_test_split (df['text'], df['topic'], test_size = 0.2 , random_state=42)


In [14]:
#embedding_mdl
model = gensim.models.Word2Vec.load('AEM_small_300')


In [15]:
# get embedding model in right format (vector array for each dictionary word)
embedding_mdl = dict(zip(model.wv.index2word, model.wv.syn0))

# count vectorizer
embedding_vect_count = embeddingvectorizer.EmbeddingCountVectorizer(embedding_mdl, 'mean')

# tfidf
embedding_vect_tfidf = embeddingvectorizer.EmbeddingTfidfVectorizer(embedding_mdl, 'mean')


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [16]:
# fit and predict
clf = LogisticRegressionCV()
pipe = make_pipeline(embedding_vect_count, clf)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarnin

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

0.8108882521489972

# Example with Keras

In [18]:
import tensorflow as tf


/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [23]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import pickle
import numpy as np

from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

import string

In [20]:
model = Word2Vec.load("AEM_small_300")
model.wv.save_word2vec_format('AEM300.txt', binary=False)
AEM = 'AEM300.txt'


In [21]:
def encodeY(Y):
    '''create one-hot (dummies) for output, encode class values as integers
    '''
    encoder = LabelEncoder()
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    dummy_y = tf.keras.utils.to_categorical(encoded_Y)
    return dummy_y

In [25]:
X_train, X_test, y_train, y_test = train_test_split([t.translate(str.maketrans('', '', string.punctuation)) for t in df['text']], encodeY(df['topic'].map(int)), test_size = 0.2)

In [26]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_train)


In [27]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(X_train)
# pad sequences
max_length = max([len(s.split()) for s in X_train])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# define training labels
#ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])

In [28]:
encoded_docs = tokenizer.texts_to_sequences(X_test)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [29]:
#Xtrain[0]

In [30]:
embeddings_index = {}
with open(AEM) as f:
    numberofwordvectors, dimensions = [int(e) for e in next(f).split()]
    for line in tqdm(f):
        values = line.split()
        embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
       # word = values[0]
       # coefs = np.asarray(values[1:], dtype='float32')
      #  embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))
print('Should be {} vectors with {} dimensions'.format(numberofwordvectors, dimensions))

957723it [01:42, 9384.11it/s]

Found 957723 word vectors.
Should be 957723 vectors with 300 dimensions


In [31]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    words_not_found = 0 
    total_words = 0
    debug = []
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, 300))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in tqdm(vocab.items()):
        e = embedding.get(word, None)
        if e is not None:   # if we do not find the word, we do not want to replace anything but leave the zero's
            weight_matrix[i] = e
            total_words+=1
        else:
            words_not_found+=1
            debug.append(word)
    print('Weight matrix created. For {} out of {} words, we did not have any embedding.'.format(words_not_found, total_words))
    return debug, weight_matrix

In [32]:
missingwords, embedding_vectors = get_weight_matrix(embeddings_index, tokenizer.word_index)

100%|██████████| 97777/97777 [00:00<00:00, 138162.44it/s]

Weight matrix created. For 25716 out of 72061 words, we did not have any embedding.


In [33]:
embedding_vectors

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.09868538, -1.55107081,  1.55730474, ..., -1.40080082,
         0.31662685, -0.86707038],
       [-2.0485208 , -1.10919297, -0.45098323, ...,  3.14795828,
        -1.99346578, -0.84098852],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.34353146, -2.99327826, -0.29971585, ...,  0.08467946,
        -0.7595104 , -2.40746951],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [34]:
len(embedding_vectors), len(Xtrain)

(97778, 2788)

In [35]:
embedding_layer = Embedding(len(tokenizer.word_index)+1, 300, weights=[embedding_vectors], input_length=max_length, trainable=False)

W0312 18:46:54.311015 140055976785664 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [36]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W0312 18:46:56.630981 140055976785664 deprecation.py:506] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 51138, 300)        29333400  
_________________________________________________________________
conv1d (Conv1D)              (None, 51134, 128)        192128    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 25567, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3272576)           0         
_________________________________________________________________
dense (Dense)                (None, 4)                 13090308  
Total params: 42,615,836
Trainable params: 13,282,436
Non-trainable params: 29,333,400
_________________________________________________________________
None


In [37]:
# alternatief model

numberoflabels = 4
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 4, activation='relu'))
model.add(MaxPooling1D(4))
model.add(MaxPooling1D(4))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=numberoflabels, activation='softmax'))   # voor twee categorien sigmoid, voor 1 tanh
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 51138, 300)        29333400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51135, 128)        153728    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12783, 128)        0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3195, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 408960)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                26173504  
_________________________________________________________________
dense_2 (Dense)              (None, 4)                

In [38]:
VALIDATION_SIZE=200

model.fit(Xtrain[:-VALIDATION_SIZE], y_train[:-VALIDATION_SIZE], 
          epochs=3, verbose=True,
          validation_data=(Xtrain[-VALIDATION_SIZE:], y_train[-VALIDATION_SIZE:]))

Train on 2588 samples, validate on 200 samples
Epoch 1/3
2588/2588 [==============================] - 1589s 614ms/sample - loss: 2.7603 - acc: 0.5804 - val_loss: 0.9312 - val_acc: 0.7100
Epoch 2/3
2588/2588 [==============================] - 805s 311ms/sample - loss: 0.6632 - acc: 0.7987 - val_loss: 0.9076 - val_acc: 0.7200
Epoch 3/3
2588/2588 [==============================] - 755s 292ms/sample - loss: 0.3572 - acc: 0.8903 - val_loss: 0.8526 - val_acc: 0.7650


In [39]:
loss, acc = model.evaluate(Xtest, y_test, verbose=True)
print('Test Accuracy: %f' % (acc*100))

698/698 [==============================] - 62s 89ms/sample - loss: 0.8811 - acc: 0.7980
Test Accuracy: 79.799426


In [40]:
# compile network
# fit network
model.fit(Xtrain, y_train, epochs=3, verbose=True)
# evaluate
loss, acc = model.evaluate(Xtest, y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))


Epoch 1/3
2788/2788 [==============================] - 791s 284ms/sample - loss: 0.2312 - acc: 0.9401
Epoch 2/3
2788/2788 [==============================] - 788s 283ms/sample - loss: 0.1504 - acc: 0.9688
Epoch 3/3
2788/2788 [==============================] - 825s 296ms/sample - loss: 0.1323 - acc: 0.9792
Test Accuracy: 79.512894


In [41]:
# embeddings_index['man'] - embeddings_index['vrouw']